In [8]:
%%time
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)
import tools.Sample_Tools as smpl
import tools.Factor_Cacher as cacher
import tools.Pretreat_Tools as pretreat
from base.JuUnits import excute_for_multidates

import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2
%aimport tools.Factor_Cacher


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Wall time: 763 µs


In [45]:
%%time
# factors = cacher.load_caches_adv(['alpha11','alpha24','alpha38','alpha41'],'2021-11-24','2021-11-24')
# codes = factors.index.get_level_values(1).unique()
# dates = factors.index.get_level_values(0).unique()
# data = smpl.get_data(codes.to_list(),dates.min().strftime('%Y-%m-%d'),dates.max().strftime('%Y-%m-%d')).data
# mv_industry = smpl.add_marketvalue_industry(data).drop(columns=data.columns.to_list())
# industy = mv_industry['industry']
cap = mv_industry['totalCapital']
adj_weight = np.sqrt(cap)/np.sqrt(cap).sum()
adj_weight_mat = pd.DataFrame(np.diag(adj_weight),index=adj_weight.index, columns=adj_weight.index)
# ret_forward = smpl.get_forward_return(data,'close')
# ret_forward.name = 'ret_forward'
# factors = factors.assign(country=1.0, industry = data['industry'])
# factors = pd.get_dummies(factors,'industry')


Wall time: 1.07 ms


date              2021-11-24                                                \
code                  000001    000002   000063    000066    000069 000100   
date       code                                                              
2021-11-24 000001    0.00619  0.000000  0.00000  0.000000  0.000000    0.0   
           000002    0.00000  0.004977  0.00000  0.000000  0.000000    0.0   
           000063    0.00000  0.000000  0.00413  0.000000  0.000000    0.0   
           000066    0.00000  0.000000  0.00000  0.002145  0.000000    0.0   
           000069    0.00000  0.000000  0.00000  0.000000  0.002378    0.0   
...                      ...       ...      ...       ...       ...    ...   
           603899    0.00000  0.000000  0.00000  0.000000  0.000000    0.0   
           603939    0.00000  0.000000  0.00000  0.000000  0.000000    0.0   
           603986    0.00000  0.000000  0.00000  0.000000  0.000000    0.0   
           603993    0.00000  0.000000  0.00000  0.000000  0.000000    0.0   
           605499    0.00000  0.000000  0.00000  0.000000  0.000000    0.0   

date                                           ...                       \
code              000157 000166 000301 000333  ... 603659 603799 603806   
date       code                                ...                        
2021-11-24 000001    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0   
           000002    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0   
           000063    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0   
           000066    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0   
           000069    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0   
...                  ...    ...    ...    ...  ...    ...    ...    ...   
           603899    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0   
           603939    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0   
           603986    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0   
           603993    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0   
           605499    0.0    0.0    0.0    0.0  ...    0.0    0.0    0.0   

date                                                                     \
code              603833 603882    603899    603939    603986    603993   
date       code                                                           
2021-11-24 000001    0.0    0.0  0.000000  0.000000  0.000000  0.000000   
           000002    0.0    0.0  0.000000  0.000000  0.000000  0.000000   
           000063    0.0    0.0  0.000000  0.000000  0.000000  0.000000   
           000066    0.0    0.0  0.000000  0.000000  0.000000  0.000000   
           000069    0.0    0.0  0.000000  0.000000  0.000000  0.000000   
...                  ...    ...       ...       ...       ...       ...   
           603899    0.0    0.0  0.002456  0.000000  0.000000  0.000000   
           603939    0.0    0.0  0.000000  0.001951  0.000000  0.000000   
           603986    0.0    0.0  0.000000  0.000000  0.003391  0.000000   
           603993    0.0    0.0  0.000000  0.000000  0.000000  0.003765   
           605499    0.0    0.0  0.000000  0.000000  0.000000  0.000000   

date                        
code                605499  
date       code             
2021-11-24 000001  0.00000  
           000002  0.00000  
           000063  0.00000  
           000066  0.00000  
           000069  0.00000  
...                    ...  
           603899  0.00000  
           603939  0.00000  
           603986  0.00000  
           603993  0.00000  
           605499  0.00292  

[255 rows x 255 columns]

In [103]:
k=factors.shape[1]
diag_r = np.diag(np.ones(k))
location = k-1
r=np.delete(diag_r,location,axis=1)

r


array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 0.]])

In [15]:
r.shape

(128, 127)

In [106]:
# factors.info(memory_usage='deep')
# factors.memory_usage(deep=True)
# import gc
# gc.collect()
# t = pd.DataFrame([[1,2,3],[4,5,6],[7,8,9]])
# t = pd.DataFrame(np.diag([1,2,3]))
# # t.div(t.iloc[-1]).iloc[:-1]
# np.delete(t.values,0,axis=1)
for i in range(1,51):
    print('test%d'% i)

test1
test2
test3
test4
test5
test6
test7
test8
test9
test10
test11
test12
test13
test14
test15
test16
test17
test18
test19
test20
test21
test22
test23
test24
test25
test26
test27
test28
test29
test30
test31
test32
test33
test34
test35
test36
test37
test38
test39
test40
test41
test42
test43
test44
test45
test46
test47
test48
test49
test50
